## Load libraries

In [1]:
%pylab inline

import pandas as pd
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os

In [3]:
from cvbi.base_imaris.objects import GetSurpassObjects

#### Give Object Type

In [4]:
import ImarisLib 

In [5]:
def GetObjectId():
    
    
    vImarisLib = ImarisLib.ImarisLib()
    vServer = vImarisLib.GetServer()
    vNumberOfObjects = vServer.GetNumberOfObjects()
    
    for vIndex in range(vNumberOfObjects):
        vObjectId = vServer.GetObjectID(vIndex)
        return vObjectId; # work with the ID (return first one)
    
    return -1 # invalid id

In [6]:
aImarisId = GetObjectId()
vImarisLib = ImarisLib.ImarisLib()
vImaris = vImarisLib.GetApplication(aImarisId)
vDataSet = vImaris.GetDataSet()

In [7]:
imaris_file = vImaris.GetCurrentFileName()
imaris_dir = os.path.dirname(imaris_file)
imaris_name = os.path.basename(imaris_file)

In [8]:
object_type = 'surfaces'
object_name = 'Th1'

## Get cell stats

In [209]:
def get_statistics_cell(vImaris , object_type , object_name):

    """
    Get All statistics from Imaris for a particular object type in current session.
    Input :

    import ImarisLib
    from cvbi.base_imaris.connection_helpers import GetObjectId

    vImarisLib = ImarisLib.ImarisLib()
    aImarisId = GetObjectId()
    vImaris = vImarisLib.GetApplication(aImarisId)

    df = get_imaris_statistics(vImaris = vImaris, object_type = 'surfaces', object_name = 'Th1')

    """

    objects = GetSurpassObjects(vImaris=vImaris, search=object_type)
    object_cells = objects[object_name]
    object_cell_stats = object_cells.GetStatistics()

    # Get individual cell IDs and track IDs to create set of edges which form a track

    cells = object_cells.GetIds()
    tracks = object_cells.GetTrackIds()
    edges_indices = object_cells.GetTrackEdges()
    edges = [[str(cells[start]),str(cells[stop])] for [start, stop] in edges_indices]

    track_cell_mapping = {}
    for trackID, (start, stop) in zip(tracks, edges_indices):

        start = cells[start]
        stop = cells[stop]

        track_cell_mapping[str(start)] = str(trackID)
        track_cell_mapping[str(stop)] = str(trackID)

    track_cell_mapping_df = pd.DataFrame.from_dict(track_cell_mapping, orient='index')
    track_cell_mapping_df.reset_index(inplace=True)
    track_cell_mapping_df.columns = ['objectID', 'trackID']

    stats_df = pd.DataFrame({'objectID': [str(objectID) for objectID in object_cell_stats.mIds],
                             'names': object_cell_stats.mNames,
                             'values': object_cell_stats.mValues})

    stats_track_df = pd.merge(left=track_cell_mapping_df,
                              right=stats_df,
                              how='inner')

    stats_pivot = stats_track_df.pivot_table(index=['trackID', 'objectID'],
                                             columns='names',
                                             values='values')
    stats_pivot_df = stats_pivot.reset_index()
    stats_pivot_df['time'] = stats_pivot_df.loc[:, 'Time Index'].values
    stats_pivot_df['track_time'] = stats_pivot_df.loc[:, 'Time Since Track Start'].values

    return(stats_pivot_df)

In [210]:
all_stats_cells = get_statistics_cell(vImaris=vImaris, object_type=object_type, object_name=object_name)

## Get track stats

In [214]:
def get_track_statistics(vImaris, object_type, object_name):

    objects = GetSurpassObjects(vImaris=vImaris, search=object_type)
    object_cells = objects[object_name]
    object_cell_stats = object_cells.GetStatistics()

    data_list = [[objectID,name,unit,value] for objectID,name,unit,value in zip(object_cell_stats.mIds,
                                                                                object_cell_stats.mNames,
                                                                                object_cell_stats.mUnits,
                                                                                object_cell_stats.mValues) if (objectID>100000)]
    data_df_long = pd.DataFrame(data_list,columns=['trackID','colname','unit','values'])
    data_df_wide = data_df_long.pivot_table(index='trackID',columns='colname').loc[:,'values'].reset_index(col_level=1)

    return( [ data_df_long, data_df_wide ] )

In [215]:
objects = GetSurpassObjects(vImaris=vImaris, search=object_type)
object_cells = objects['Th1']
object_cell_stats = object_cells.GetStatistics()

condition = [name[:5]=='Track' for name in object_cell_stats.mNames]

data_list = [[objectID,name,unit,value] for objectID,name,unit,value in zip(object_cell_stats.mIds,
                                                                            object_cell_stats.mNames,
                                                                            object_cell_stats.mUnits,
                                                                            object_cell_stats.mValues) if (objectID>100000)]
data_df_long = pd.DataFrame(data_list,columns=['trackID','colname','unit','values'])
data_df_wide = data_df_long.pivot_table(index='trackID',columns='colname').loc[:,'values'].reset_index(col_level=1)

In [216]:
data_df_long,data_df_wide = get_track_statistics(vImaris=vImaris, object_type=object_type, object_name=object_name)

In [145]:
data_df_wide.to_csv('./sample.csv', index=False)